# Exploring bitcoin fluctuations

There are two questions that we'll address in this analysis:

* what factors do you think led to bitcoin’s boom in 2017?
* how did this affect other cryptocurrencies?
* Do you have any speculations about the future?

To complete this analysis I have downloaded two datasets from Yahoo Finance.

* Historical price data about BTC-USD can be found [here](https://finance.yahoo.com/quote/BTC-USD/history?p=BTC-USD)
* Historical price data about ETH-USD can be found [here](https://finance.yahoo.com/quote/ETH-USD/history?period1=1434931200&period2=1592784000&interval=1d&filter=history&frequency=1d)

For this analysis, I'll look at price data from a specific range of dates. Furthermore, I'll compare the market dynamics between bitcoin and ethereum since according to [this articile](https://www.forbes.com/sites/bernardmarr/2017/12/06/a-short-history-of-bitcoin-and-crypto-currency-everyone-should-read/#14c3403c3f27) it was - in the time inteval we are interested in - a potential contender against bitcoin.

## 1. Opening up and reading the data

In [1]:
# start by creating the dependencies
import numpy as np
import datetime as dt
from csv import reader


In [2]:
# defining a function to open up the files
def open_csv(file_name):
    file_name = open(file_name)
    dataset_raw = list(reader(file_name))
    return dataset_raw

# opening both datasets
bitcoin_data_raw = open_csv("BTC-USD.csv")
ethereum_data_raw = open_csv("ETH-USD.csv")

In [3]:
# exploding the bitcoin dataset
bitcoin_data_raw[0:3]

[['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume'],
 ['2015-06-22',
  '243.968994',
  '247.917007',
  '243.779007',
  '246.990005',
  '246.990005',
  '17692500'],
 ['2015-06-23',
  '246.927002',
  '247.304001',
  '243.132996',
  '244.296005',
  '244.296005',
  '15108700']]

In [4]:
# exploring the ethereum dataset
ethereum_data_raw[0:3]

[['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume'],
 ['2015-08-07',
  '2.831620',
  '3.536610',
  '2.521120',
  '2.772120',
  '2.772120',
  '164329'],
 ['2015-08-08',
  '2.793760',
  '2.798810',
  '0.714725',
  '0.753325',
  '0.753325',
  '674188']]

Based on the exploration of the data, we can see that:

* the structure of the headers is the same
* the earliest dat that would allow us to compare the datasets is 7th August 2015
* The format for the date is year-month-day

For this analysis, we are going to focus our attention to closing price data from the period of September 2015 to the 31 December 2017.

## 2. Data cleaning

At this point we are going to:

* transform the dates into date objects from the date column
* extract only the relevant entries that go from the 1 September 2015 to the 31 December 2017
* extract only the relevant columns for the analysis


In [5]:
# creating a function to transform the dates into datetime objects

def from_date_to_datetime(input_list, input_index):
    modified_list = []
    date_format = "%Y-%m-%d"
    for element in input_list:
        date_string = element[input_index]
        datetime_object = dt.datetime.strptime(date_string,date_format)
        element[input_index] = datetime_object
        modified_list.append(element)
    return modified_list

# cleaning the dates in the dataset. Iterating over the set and excluding the header rows
bitcoin_cleaned_dates = from_date_to_datetime(bitcoin_data_raw[1:],0)
etheremum_cleaned_dates = from_date_to_datetime(ethereum_data_raw[1:],0)        

In [6]:
# setting up the endpoints for the time interval
start_date = dt.datetime(2015,9,1)
end_date = dt.datetime(2017,12,31)

# creating a function to filter the entries based on the date interval and extract the colums we are interested in working
def extract_dataset(input_list):
    cleaned_list = []
    for element in input_list:
        element_date = element[0]
        # checking whether the element data is in the time interval
        if element_date > start_date and element_date < end_date:
            # extracting the data from the designed colums: date, closing price, volume
            cleaned_element = [element[0],float(element[4]),int(element[-1])]
            cleaned_list.append(cleaned_element)
    return cleaned_list

# calling the function on bitcoin and ethereum data
bitcoin_clean = extract_dataset(bitcoin_cleaned_dates)
ethereum_clean = extract_dataset(etheremum_cleaned_dates)

In [7]:
# exploring cleaned bitcoin data
bitcoin_clean[0:5]

NameError: name 'bitcoin' is not defined

In [8]:
# exploring cleaned ethereum data
ethereum_clean[0:5]

NameError: name 'ethereum' is not defined

To help with the analysis, I'm going to convert the list of list into a nd-array using the numpy module. This will help me speed up the navigation, selection and exploration of the datasets.

In [9]:
# transforming the cleaned list of lists into nd-arrays 
bitcoin = np.array(bitcoin_clean)
ethereum = np.array(ethereum_clean)

In [10]:
bitcoin[0:5]

array([[datetime.datetime(2015, 9, 2, 0, 0), 229.283997, 18760400],
       [datetime.datetime(2015, 9, 3, 0, 0), 227.182999, 17482000],
       [datetime.datetime(2015, 9, 4, 0, 0), 230.298004, 20962400],
       [datetime.datetime(2015, 9, 5, 0, 0), 235.018997, 20671400],
       [datetime.datetime(2015, 9, 6, 0, 0), 239.839996, 25473700]],
      dtype=object)

In [11]:
ethereum[0:5]

array([[datetime.datetime(2015, 9, 2, 0, 0), 1.29479, 883769],
       [datetime.datetime(2015, 9, 3, 0, 0), 1.26493, 702991],
       [datetime.datetime(2015, 9, 4, 0, 0), 1.27441, 474391],
       [datetime.datetime(2015, 9, 5, 0, 0), 1.33881, 448453],
       [datetime.datetime(2015, 9, 6, 0, 0), 1.29583, 491968]],
      dtype=object)

## 2. Analyzing the data

At this point, we can perform some descriptive statistics operations on the cleaned datasets and compare the dynamics between bitcoin and ethereum.

In [12]:
# Since I'll be running the same operations for each cryptocurrency, I'll create a function to explore and read the data
def explore_cryptocurrency(input_array, input_currency):
    # gathering price information
    crypto_open = input_array[0,1]
    crypto_close = input_array[-1,1]
    cypto_range = crypto_close - crypto_open
    crypto_multiplier = crypto_close/crypto_open
    crypto_max = input_array[:,1].max()
    crypto_min = input_array[input_array[:,1] > 0,1].min()
    crypto_mean = input_array[:,1].mean()
    crypto_maxdrawdown = crypto_max - crypto_min

    # gather volume information
    crypto_volume_open = input_array[0,2]
    crypto_volume_close = input_array[-1,2]
    cypto_volume_range = crypto_close - crypto_open
    crypto_volume_max = input_array[:,2].max()
    crypto_volume_min = input_array[input_array[:,2] > 0,2].min()
    crypto_volume_mean = input_array[:,2].mean()
    crypto_volume_maxdrawdown = crypto_volume_max - crypto_volume_min

    # outputting the information
    print("{} opened at: USD {:,.2f}".format(input_currency, crypto_open))
    print("{} closed at: USD {:,.2f}".format(input_currency, crypto_close))
    print("{}'s price range was of USD {:,.2f}. With a max of USD {:,.2f} and a minimum of USD {:,.2f} ".format(input_currency, cypto_range, crypto_max, crypto_min))
    print("It grew at a multiplier of {:.1f}x".format(crypto_multiplier))
    print("In terms of volumes we've seen an open of {:,} and a close of {:,} with a peak at {:,} and a minimum of {:,}. The maximum volume drawdown was {:,}".format(crypto_volume_open,crypto_volume_close,crypto_volume_max,crypto_volume_min,crypto_volume_maxdrawdown))




In [13]:
# exploring market data about bitcoin
explore_cryptocurrency(bitcoin,"bitcoin")

bitcoin opened at: USD 229.28
bitcoin closed at: USD 12,952.20
bitcoin's price range was of USD 12,722.92. With a max of USD 19,497.40 and a minimum of USD 227.09 
It grew at a multiplier of 56.5x
In terms of volumes we've seen an open of 18,760,400 and a close of 14,452,599,808 with a peak at 22,197,999,616 and a minimum of 12,712,600. The maximum volume drawdown was 22,185,287,016


In [14]:
# exploring market data about ethereum
explore_cryptocurrency(ethereum, "ethereum")

ethereum opened at: USD 1.29
ethereum closed at: USD 717.26
ethereum's price range was of USD 715.96. With a max of USD 826.82 and a minimum of USD 0.43 
It grew at a multiplier of 554.0x
In terms of volumes we've seen an open of 883,769 and a close of 3,187,780,096 with a peak at 5,179,829,760 and a minimum of 102,128. The maximum volume drawdown was 5,179,727,632


## Conclusion

From this short analysis, looking at the period between September 2015 and December 2017, we can see that there has been a strong price increase for both currencies. As for the causes of the increase in price, at this stage I don't have enough information to model the issue correctly and pinpoint the specific causes for the increase in both price and trading volume.

I would speculate that maybe the [increasing insterest from big institutional players](https://www.coindesk.com/8-banking-giants-bitcoin-blockchain) such as Barclays, BNP Paribas, or Citi Bank have increased the demand for cryptocurrencies. There are players that have even created their own [trading desk for bitcoins](https://www.cnbc.com/2017/12/21/goldman-sachs-launching-trading-desk-for-bitcoin-report-says.html). If players of this size enter a specific market we can foresee an increase in market action and a strong increase in price. This is just a hypothesis.

As for the second question, as we can see from the data, the growing interest and validation that bitcoin generated to the cryptomarkets has had a positive effect on other cryptocurrencies. Although, bitcoin grew by a multiple of 56.5, ethereum - during the same span of time grew 554x.

From my perspective the future is uncertain. We could see a growth activity and interest from more institutional and retail investors, as well as growing volume of crypto based commerce. Hence driving the market action (price and volumes) to an even stronger growth. However, there are questions about the stability of cryptocurrencies - both from an economic perspective (such as the difficulties involved in performing fundamental analysis on cryptocurrencies) as well as technological perspective (like the question, is there a point where the size of the distributed ledger and processing power needed to manage transactions efficiently exceeds our current computing resources available). These variables introduce an element of uncertainty regaring the fate and future of cryptocurrencies.
